In [103]:
import inferelator_ng.prior as priors
import os
import pandas as pd

In [104]:
base_dir='/Users/ndeveaux/Dropbox (Simons Foundation)/atac_parameter_search/ChipSeq/chipseq_priors/chipseq_mm10_th17_prior_gen_2017_01/references'

In [105]:
macs_folders = ['2016_11_28_MACS_out', '2016_11_15_MACS_out']

In [106]:
metadata_file = os.path.join(base_dir, 'metadata_ChIP.txt')
metadata = pd.read_csv(metadata_file, sep = '\t')

In [107]:
gene_bed = os.path.join(base_dir, 'tss_mm10.bed')
dist=10000

In [108]:
gene_df = pd.read_csv(gene_bed, sep = '\t', header=None)

In [109]:
priors = {}

In [ ]:
combined_prior = pd.dataframe

In [ ]:
for row in metadata.iterrows():
    row_content = row[1]
    sample = ('_').join([row_content['sampleName'], row_content['compare2']])
    for mac in macs_folders:  
        motif_bed = os.path.join(os.path.join(os.path.join(base_dir, mac), sample), '_'.join([sample, 'peaks.bed']))
        p = Prior(motif_bed, gene_bed, regulators = [row_content['gene']], targets = list(gene_df[3]), max_distance=dist, single_tf=True)
        priors['_'.join([mac, sample])] = p.make_prior()

In [101]:
# Sanity check: see if prior is what's expected, 7. 
max(priors['2016_11_15_MACS_out_SL1040_SL972']['Stat3'])

7

In [84]:
gene_df[gene_df[3] == 'uc011whu.1_up_1_chr1_4858327_f']

,0,1,2,3,4,5
18,chr1,4858326,4858327,uc011whu.1_up_1_chr1_4858327_f,0,+


In [110]:
new_prior = priors['2016_11_15_MACS_out_SL1040_SL972']
new_prior[new_prior['Stat3'] > 6]

,TF
uc057alc.1_up_1_chr8_35620774_f,7
uc009mmr.2_up_1_chr8_84662853_r,7
uc009nyw.2_up_1_chr8_126593437_r,7
uc011ybs.1_up_1_chr11_86584159_r,7


In [102]:
metadata

,gene,sampleName,compare2
0,Batf,SL3033,SL3036
1,Batf,SL3037,SL3036
2,Batf,SL10570,SL10564
3,Maf,SL3031,SL2871
4,Maf,SL3032,SL2871
5,Ctcf,SL3034,SL3035
6,Ctcf,SL3783,SL3782
7,Etv6,SL13368,SL13364
8,Etv6,SL10573,SL10567
9,Fosl2,SL6498,SL6497


In [97]:
motif_bed='/Users/ndeveaux/Dropbox (Simons Foundation)/atac_parameter_search/ChipSeq/2016_11_22_performance_test_output/SL1040_SL972_peaks.bed'